In [86]:
# !pip install ray
# !pip install "ray[default]"
# !pip uninstall grpcio
#!pip install lxml
import requests
from time import time 
from bs4 import BeautifulSoup
from tqdm import tqdm
import ray
basic_url = "https://www.10000recipe.com"
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
ingridient_seperator = "                                                                                    "

In [ ]:
# ray


In [82]:
# lxml 사용시 htmlParseEntityRef: expecting ';' warning 뜸
# 실행은 되지만 warning 문을 출력해서 log에 뜸
# 그래서 html.parser 사용

In [137]:
@ray.remote
def crawling_recipe_path_list(page):
    recipe_path_list = []
    url = f"{basic_url}/recipe/list.html?order=date&page={page}"
    response = requests.get(url, headers = headers)
    # dag만들 때 오류처리 하기
    if response.status_code !=200:
        print(f"{page} 페이지 접속 불가")
        return []
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        for a in soup.find("ul","common_sp_list_ul").find_all("a","common_sp_link"):
            recipe_path_list.append(a.get("href"))
    except:
        print(f"{page}페이지 에러 뜸 / 빈 리스트 반환")
        return []
        
    return recipe_path_list

In [3]:
# try catch로 오류 처리 해주기
def parsing_recipe_ingredients(soup):
    ingredient_list = []
    for ul in (soup.find("div", "ready_ingre3").find_all("ul")):
        for li in ul.find_all("li"):
            ingredient = {}
            ingredient['name'] = (li.text.split("\n")[0].split(ingridient_seperator)[0].strip(" "))
            ingredient['quantity'] = (li.find("span","ingre_unit").text.strip(" "))
            ingredient_list.append(ingredient)
    return ingredient_list

In [4]:
def parsing_recipe_sequences(soup):
    sequence_list = []
    for idx, div in enumerate(soup.find("div","view_step").find_all("div","view_step_cont")):
        sequence = {}
        sequence['sequence'] = idx+1
        sequence['content'] = div.text
        sequence_list.append(sequence)
    return sequence_list

In [144]:
@ray.remote
def makeing_recipe(recipe_path):
    url = basic_url+recipe_path
    response = requests.get(url, headers=headers)
    # dag만들 때 오류처리 하기
    if response.status_code !=200:
        pass
    soup = BeautifulSoup(response.text, "html.parser")
    
    recipe = {}
    recipe['recipe_id'] = recipe_path.split("/")[2]
    try:
        recipe['title'] = soup.find("div","view2_summary st3").find("h3").text
    except:
        print(f"이름 오류 {recipe_path}")
        
    try:
        recipe['size'] = soup.find("span","view2_summary_info1").text
    except:
        print(f"몇인분 오류 {recipe_path}")
        
    try:
        recipe['ingredients'] = parsing_recipe_ingredients(soup)
    except:
        print(f"재료 오류 {recipe_path}")
        
    try:
        recipe['sequences'] = parsing_recipe_sequences(soup)
    except:
        print(f"조리법 오류 {recipe_path}")
    
    return recipe

In [95]:
ray.init()

2022-07-27 18:45:57,254	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8266


RayContext(dashboard_url='127.0.0.1:8266', python_version='3.9.13', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-07-27_18-45-55_565745_71918/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-07-27_18-45-55_565745_71918/sockets/raylet', 'webui_url': '127.0.0.1:8266', 'session_dir': '/tmp/ray/session_2022-07-27_18-45-55_565745_71918', 'metrics_export_port': 62588, 'gcs_address': '127.0.0.1:62975', 'address': '127.0.0.1:62975', 'node_id': '6acc21d48a08371b0ae2a5ae52672b9efa4f9a3f261d4360b9cdf622'})

In [139]:
#옛날건 어떤가 확인해보기 레시피 크롤링
start = time()
recipe_path_list = ray.get(crawling_recipe_path_list.remote(1123))
print(time()-start)

0.31472206115722656


In [140]:
start = time()
ref_list = []
for recipe_path in tqdm(recipe_path_list):
    ref_list.append(makeing_recipe.remote(recipe_path))
recipe_list = ray.get(ref_list)
c = (time()-start)
print(c)

100%|█████████████████████████████████████████| 40/40 [00:00<00:00, 8250.82it/s]


(makeing_recipe pid=74539) 이름 오류 https://www.10000recipe.com/recipe/6936518
(makeing_recipe pid=74539) 몇인분 오류 https://www.10000recipe.com/recipe/6936518
(makeing_recipe pid=74539) 재료 오류 https://www.10000recipe.com/recipe/6936518
(makeing_recipe pid=74539) 조리법 오류 https://www.10000recipe.com/recipe/6936518
1.3891141414642334


In [145]:
for i in recipe_list:
    if len(i.keys()) ==1 :
        print(i)

{'recipe_id': '6936518'}


In [146]:
ray.shutdown()